<a href="https://colab.research.google.com/github/SamiraHabib/Coursera_Capstone/blob/main/Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursera Capstone #

# Importando as bibliotecas #

In [7]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 81kB 5.0MB/s 
  Created wheel for folium: filename=folium-0.5.0-cp37-none-any.whl size=76240 sha256=804aa19406d84bbb16ed7cff17d5544705af6e9a3da56c0256eafd828ceaaf03
  Stored in directory: /root/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.5.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
Folium installed
Libraries imported.


# Definição das credenciais e version do Foursquare #

In [8]:
CLIENT_ID = '1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA' # your Foursquare ID
CLIENT_SECRET = 'ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D' # your Foursquare Secret
ACCESS_TOKEN = '' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA
CLIENT_SECRET:ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D


# Endereço do local escolhido #

In [9]:
#Exemplo - alterar
address = 'Rio de Janeiro, RJ'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-22.9110137 -43.2093727


# Localização #

In [10]:
#Localidade exemplo, mudar
latitude = -22.9110137
longitude = -43.2093727

# Define URL #

In [11]:
search_query = 'colégio'
radius = 1500
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

colégio .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=1LFBJCCEY1TSE4R1J4Y2MX5PZMPVS4O4IFXTHHCQQMYT53SA&client_secret=ZL54S51I5OLJWENEGTRHCTKLYQ21GIRDXWCLZUMYS24J4J2D&ll=-22.9110137,-43.2093727&v=20180604&query=colégio&radius=1500&limit=50'

# Envia uma solicitação GET e examina os resultados #

In [12]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '608233ccff2af23a98d8c421'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/default_',
       'suffix': '.png'},
      'id': '52e81612bcbc57f1066b7a46',
      'name': 'Private School',
      'pluralName': 'Private Schools',
      'primary': True,
      'shortName': 'Private School'}],
    'hasPerk': False,
    'id': '4e60f0fb315114bf603f914f',
    'location': {'address': 'Tv. Dr. Araújo, 21',
     'cc': 'BR',
     'city': 'Rio de Janeiro',
     'country': 'Brasil',
     'distance': 691,
     'formattedAddress': ['Tv. Dr. Araújo, 21',
      'Rio de Janeiro, RJ',
      '20270-070',
      'Brasil'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -22.914814836666505,
       'lng': -43.21470669286932}],
     'lat': -22.914814836666505,
     'lng': -43.21470669286932,
     'postalCode': '20270-070',
     'state': 'RJ'},
    'name': 'CARJ - Colégio Adventista do Rio de Janeiro',
   

## Pegando a parte relevante do JSON e transformando em um dataframe (pandas) ##

In [13]:
#transformando em dataframe
venues = results['response']['venues']
df_school = json_normalize(venues)
df_school.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4e60f0fb315114bf603f914f,CARJ - Colégio Adventista do Rio de Janeiro,"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",v-1619145676,False,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",NaN
1,4d30474405585941fce35b37,Colégio Qi,"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",v-1619145676,False,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",NaN
2,4c7cde823e90a1cda18743be,Pensi Colégio e Curso,"[{'id': '52e81612bcbc57f1066b7a46', 'name': 'P...",v-1619145676,False,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",Unidade Tijuca I
3,4c2b56d0b34ad13a8293e9ce,Colégio e Curso pH,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",v-1619145676,False,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",NaN
4,53e9e823498ecddee79da976,Colegio e curso superior,"[{'id': '4bf58dd8d48988d1ae941735', 'name': 'U...",v-1619145676,False,Tijuca,-22.905207,-43.205631,"[{'label': 'display', 'lat': -22.9052066802978...",751,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Tijuca, Rio de Janeiro, RJ, Brasil]",NaN


# Define information of interest and filter dataframe #

In [14]:
# Mantém somente as colunas que incluem venue name e qualquer coisa que é associada com a localização
filtered_columns = ['name', 'categories'] + [col for col in df_school.columns if col.startswith('location.')] + ['id']
df_school_filtered = df_school.loc[:, filtered_columns]

# função que extrai a categoria de venue 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filtra a categoria por cada linha do data frame
df_school_filtered['categories'] = df_school_filtered.apply(get_category_type, axis=1)

# limpa os nomes das colunas para manter somente o último termo
df_school_filtered.columns = [column.split('.')[-1] for column in df_school_filtered.columns]

df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,CARJ - Colégio Adventista do Rio de Janeiro,Private School,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",NaN,4e60f0fb315114bf603f914f
1,Colégio Qi,Private School,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",NaN,4d30474405585941fce35b37
2,Pensi Colégio e Curso,Private School,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",Unidade Tijuca I,4c7cde823e90a1cda18743be
3,Colégio e Curso pH,High School,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",NaN,4c2b56d0b34ad13a8293e9ce
4,Colegio e curso superior,University,Tijuca,-22.905207,-43.205631,"[{'label': 'display', 'lat': -22.9052066802978...",751,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Tijuca, Rio de Janeiro, RJ, Brasil]",NaN,53e9e823498ecddee79da976


# Nome e categorias dos dados #

In [15]:
df_school_filtered.columns

Index(['name', 'categories', 'address', 'lat', 'lng', 'labeledLatLngs',
       'distance', 'postalCode', 'cc', 'city', 'state', 'country',
       'formattedAddress', 'crossStreet', 'id'],
      dtype='object')

In [16]:
df_school_filtered.name

0     CARJ - Colégio Adventista do Rio de Janeiro
1                                      Colégio Qi
2                           Pensi Colégio e Curso
3                              Colégio e Curso pH
4                        Colegio e curso superior
5                            Colégio Guanabarense
6                           colégio barbara Otoni
7                       Colégio Bezerra de Araújo
8                               Colégio Curso SEI
9                              Colégio Modelo GPI
10          Colégio Miguel Couto - Unidade Tijuca
11                                 Colégio Sagres
12                         Colegio L'Hermitage RJ
13               Colégio Estadual Hebert de Souza
14                               Colegio Van Gogh
15                           Colégio Maria Raythe
16                     Colégio Mérito D' Martonne
17              Colégio Estadual Herbert De Souza
18                            Colégio Graham Bell
19                          COLÉGIO PREIRA PASSOS


In [17]:
df_school_filtered.categories

0                      Private School
1                      Private School
2                      Private School
3                         High School
4                          University
5        General College & University
6                      Student Center
7     College Administrative Building
8                              School
9                      Private School
10                     Student Center
11                     Private School
12                               None
13                             School
14                               None
15                             School
16                      Middle School
17              College Arts Building
18                  College Classroom
19                        High School
20                        High School
21                        High School
22          College Academic Building
23                        College Lab
24                            Library
25                     Private School
26          

# Identificar as ids de escolas diferentes #

In [18]:
df_school_filtered[['id', 'name']]

,id,name
0,4e60f0fb315114bf603f914f,CARJ - Colégio Adventista do Rio de Janeiro
1,4d30474405585941fce35b37,Colégio Qi
2,4c7cde823e90a1cda18743be,Pensi Colégio e Curso
3,4c2b56d0b34ad13a8293e9ce,Colégio e Curso pH
4,53e9e823498ecddee79da976,Colegio e curso superior
5,4ca8bfd0b7106dcbc1ab75a5,Colégio Guanabarense
6,4fc4ba7fe4b084616fe44cf0,colégio barbara Otoni
7,53650bd3498eb12aad8f27c4,Colégio Bezerra de Araújo
8,5895d3afd7b4732925b68b2e,Colégio Curso SEI
9,56b20d4638fa948b56fe34b1,Colégio Modelo GPI


# Visualizar escolas no mapa ao redor da localização anteriormente citada #

In [19]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # gerar mapa centrado no entorno do RJ

# Circulo vermelho no centro
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='School',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# Escolas em azul ao entorno
for lat, lng, label in zip(df_school_filtered.lat, df_school_filtered.lng, df_school_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
       # popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

# Avaliando os dados: #

In [20]:
df_school_filtered.dtypes

name                 object
categories           object
address              object
lat                 float64
lng                 float64
labeledLatLngs       object
distance              int64
postalCode           object
cc                   object
city                 object
state                object
country              object
formattedAddress     object
crossStreet          object
id                   object
dtype: object

In [21]:
df_school_filtered.describe()

,lat,lng,distance
count,31.000000,31.000000,31.000000
mean,-22.914291,-43.215961,1264.258065
std,0.007457,0.006476,293.431306
min,-22.925018,-43.223947,691.000000
25%,-22.920109,-43.220135,1103.000000
50%,-22.914815,-43.218606,1314.000000
75%,-22.912421,-43.210980,1498.000000
max,-22.897638,-43.194500,1763.000000


In [22]:
df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,CARJ - Colégio Adventista do Rio de Janeiro,Private School,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",NaN,4e60f0fb315114bf603f914f
1,Colégio Qi,Private School,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",NaN,4d30474405585941fce35b37
2,Pensi Colégio e Curso,Private School,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",Unidade Tijuca I,4c7cde823e90a1cda18743be
3,Colégio e Curso pH,High School,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",NaN,4c2b56d0b34ad13a8293e9ce
4,Colegio e curso superior,University,Tijuca,-22.905207,-43.205631,"[{'label': 'display', 'lat': -22.9052066802978...",751,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Tijuca, Rio de Janeiro, RJ, Brasil]",NaN,53e9e823498ecddee79da976


# Número de nulos das colunas #

In [23]:
print("Number of NaN values for the column city :", df_school_filtered['city'].isnull().sum())
print("Number of NaN values for the column state :", df_school_filtered['state'].isnull().sum())
print("Number of NaN values for the column crossStreet :", df_school_filtered['crossStreet'].isnull().sum())

Number of NaN values for the column city : 9
Number of NaN values for the column state : 9
Number of NaN values for the column crossStreet : 28


In [24]:
df_school_filtered['state'].replace(np.nan,"RJ", inplace=True)

In [25]:
df_school_filtered['city'].replace(np.nan,"Rio de Janeiro", inplace=True)

In [26]:
print("Number of NaN values for the column state :", df_school_filtered['state'].isnull().sum())
print("Number of NaN values for the column city :", df_school_filtered['city'].isnull().sum())

Number of NaN values for the column state : 0
Number of NaN values for the column city : 0


In [27]:
df_school_filtered['categories'].value_counts()

Private School                     7
High School                        5
School                             4
Student Center                     2
Middle School                      1
General College & University       1
College Lab                        1
College Academic Building          1
College Auditorium                 1
University                         1
College Arts Building              1
College Administrative Building    1
College Classroom                  1
College Library                    1
Library                            1
Name: categories, dtype: int64

In [29]:
df_school_filtered.isnull().sum()

name                 0
categories           2
address              8
lat                  0
lng                  0
labeledLatLngs       0
distance             0
postalCode          15
cc                   0
city                 0
state                0
country              0
formattedAddress     0
crossStreet         28
id                   0
dtype: int64

In [30]:
df_school_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,CARJ - Colégio Adventista do Rio de Janeiro,Private School,"Tv. Dr. Araújo, 21",-22.914815,-43.214707,"[{'label': 'display', 'lat': -22.9148148366665...",691,20270-070,BR,Rio de Janeiro,RJ,Brasil,"[Tv. Dr. Araújo, 21, Rio de Janeiro, RJ, 20270...",NaN,4e60f0fb315114bf603f914f
1,Colégio Qi,Private School,"R. Ibituruna, 37",-22.913802,-43.220259,"[{'label': 'display', 'lat': -22.9138020434751...",1158,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 37, Rio de Janeiro, RJ, 20271-0...",NaN,4d30474405585941fce35b37
2,Pensi Colégio e Curso,Private School,"R. Ibituruna, 27",-22.913970,-43.219829,"[{'label': 'display', 'lat': -22.9139701021976...",1121,20271-021,BR,Rio de Janeiro,RJ,Brasil,"[R. Ibituruna, 27 (Unidade Tijuca I), Rio de J...",Unidade Tijuca I,4c7cde823e90a1cda18743be
3,Colégio e Curso pH,High School,"R. Prof. Gabizo, 211",-22.917880,-43.221191,"[{'label': 'display', 'lat': -22.9178796014997...",1432,20271-064,BR,Rio de Janeiro,RJ,Brasil,"[R. Prof. Gabizo, 211, Rio de Janeiro, RJ, 202...",NaN,4c2b56d0b34ad13a8293e9ce
4,Colegio e curso superior,University,Tijuca,-22.905207,-43.205631,"[{'label': 'display', 'lat': -22.9052066802978...",751,NaN,BR,Rio de Janeiro,RJ,Brasil,"[Tijuca, Rio de Janeiro, RJ, Brasil]",NaN,53e9e823498ecddee79da976


In [34]:
df_school_filtered['name'].count()

31

In [35]:
print('There are {} uniques categories.'.format(len(df_school_filtered['categories'].unique())))

There are 16 uniques categories.


In [39]:
# one hot encoding
df_school_onehot = pd.get_dummies(df_school_filtered[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_school_onehot['name'] = df_school_filtered['name'] 

# move neighborhood column to the first column
fixed_columns = [df_school_onehot.columns[-1]] + list(df_school_onehot.columns[:-1])
df_school_onehot = df_school_onehot[fixed_columns]

df_school_onehot.head()

,name,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Classroom,College Lab,College Library,General College & University,High School,Library,Middle School,Private School,School,Student Center,University
0,CARJ - Colégio Adventista do Rio de Janeiro,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Colégio Qi,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,Pensi Colégio e Curso,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Colégio e Curso pH,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Colegio e curso superior,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [40]:
df_school_onehot.shape

(31, 16)